In [ ]:
!pip install scikeras

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
import tensorflow as tf
import random

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
merged_n = pd.read_csv(r'/content/drive/MyDrive/Dissertation_10862121/merged_n.csv')

In [ ]:
save_dir = "r'/content/drive/MyDrive/Dissertation_10862121/"

In [ ]:
import os
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
merged_n['GMT'] = pd.to_datetime(merged_n['GMT'])
merged_n['Date'] = pd.to_datetime(merged_n['Date'])
merged_n.set_index('GMT', inplace=True)

In [ ]:
merged_n

,Total Household Consumption (N),Date,Time,Year,Month,Day,Hour,DayOfWeek,IsHoliday,Hourly Global Radiation (KJ/m2),Hourly Maximum Gust (kn),Hourly Mean Wind Direction (o),Hourly Mean Windspeed (kn),Hourly Pressure at Mean Sea Level (hPa),Hourly Rainfall Total (mm),Hourly Relative Humidity (%),Hourly Snow Depth (cm),Hourly Temperature (C),Hourly Total Cloud Cover (oktas)\n,Hourly Visibility (dm)
GMT,,,,,,,,,,,,,,,,,,,,
2013-01-01 00:00:00,1657.483601,2013-01-01,00:00:00,2013,1,1,0,1,True,0.0,13.2,232.0,7.8,999.400,0.00,92.085714,0.0,8.914286,8.000000,2133.333333
2013-01-01 01:00:00,1422.045237,2013-01-01,01:00:00,2013,1,1,1,1,True,0.0,13.6,236.0,8.0,999.675,0.00,91.042857,0.0,8.785714,8.000000,1766.666667
2013-01-01 02:00:00,1200.551314,2013-01-01,02:00:00,2013,1,1,2,1,True,0.0,13.2,246.0,7.6,999.975,0.20,92.685714,0.0,8.514286,8.000000,1333.333333
2013-01-01 03:00:00,1061.568405,2013-01-01,03:00:00,2013,1,1,3,1,True,0.0,18.6,286.0,9.0,1000.725,0.96,89.871429,0.0,7.285714,7.666667,4100.000000
2013-01-01 04:00:00,1000.474013,2013-01-01,04:00:00,2013,1,1,4,1,True,0.0,15.4,278.0,7.4,1001.300,0.08,88.585714,0.0,6.700000,4.000000,5500.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,2979.858073,2013-12-31,19:00:00,2013,12,31,19,1,False,0.0,12.8,192.0,7.8,1002.900,0.00,85.385714,0.0,5.942857,6.333333,3666.666667
2013-12-31 20:00:00,2700.407931,2013-12-31,20:00:00,2013,12,31,20,1,False,0.0,13.6,196.0,7.4,1003.175,0.00,84.442857,0.0,5.442857,1.333333,4000.000000
2013-12-31 21:00:00,2447.194261,2013-12-31,21:00:00,2013,12,31,21,1,False,0.0,13.6,186.0,7.6,1003.450,0.00,83.257143,0.0,5.414286,1.000000,3833.333333


Feature Selection

In [ ]:
merged_n = merged_n.drop(columns=["Hourly Mean Windspeed (kn)"])

Feature Engineering

Lags

Consumption

In [ ]:
#Lag for the hour before
merged_n['lag_1_hour_energy'] = merged_n['Total Household Consumption (N)'].shift(1)

#Lag for the same hour of the day before
merged_n['lag_1_day_energy'] = merged_n['Total Household Consumption (N)'].shift(24)

#Lag for the same hour of the same day of the week before (7 days before)
merged_n['lag_7_day_energy'] = merged_n['Total Household Consumption (N)'].shift(24 * 7)

Temperature

In [ ]:
#Lag for the hour before
merged_n['lag_1_hour_temp'] = merged_n['Hourly Temperature (C)'].shift(1)
merged_n['lag_1_hour_temp'] = merged_n['Hourly Temperature (C)'].shift(24)
merged_n['lag_1_hour_temp'] = merged_n['Hourly Temperature (C)'].shift(24*7)

Radiation

In [ ]:
#Lag for the hour before
merged_n['lag_1_hour_rad'] = merged_n['Hourly Global Radiation (KJ/m2)'].shift(1)
merged_n['lag_1_hour_rad'] = merged_n['Hourly Global Radiation (KJ/m2)'].shift(24)
merged_n['lag_1_hour_rad'] = merged_n['Hourly Global Radiation (KJ/m2)'].shift(24*7)

Rolling Statistics - Consumption

In [ ]:
window_size = 72
merged_n['rolling_mean_consumption'] = merged_n['Total Household Consumption (N)'].rolling(window=window_size).mean()

Cyclical Features

In [ ]:
merged_n['hour_sin'] = np.sin(2 * np.pi * merged_n['Hour'] / 24)
merged_n['hour_cos'] = np.cos(2 * np.pi * merged_n['Hour'] / 24)

Weather Combination Variables

In [ ]:
merged_n['temp_humidity_interaction'] = merged_n['Hourly Temperature (C)'] * merged_n['Hourly Relative Humidity (%)']

In [ ]:
merged_n['temp_rad_interaction'] = merged_n['Hourly Temperature (C)'] * merged_n['Hourly Global Radiation (KJ/m2)']

Weighted Observations (More weight to more recent)

In [ ]:
alpha = 0.9
merged_n['ewm_consumption'] = merged_n['Total Household Consumption (N)'].ewm(alpha=alpha).mean()

Difference between consecutive hours

In [ ]:
merged_n['consumption_diff'] = merged_n['Total Household Consumption (N)'].diff()

Dropping Missing Value Rows after Feature Engineering

In [ ]:
merged_n.dropna(inplace=True)

In [ ]:
merged_n['IsHoliday'] = merged_n['IsHoliday'].astype(int)

In [ ]:
non_num_columns = merged_n.select_dtypes(exclude=['int64', 'float64']).columns
print(non_num_columns)

Index(['Date', 'Time'], dtype='object')


In [ ]:
#Dropping columns that aren't numerical (needed for model input)
merged_n= merged_n.drop(columns=['Date', 'Time'])

In [ ]:
non_num_columns = merged_n.select_dtypes(exclude=['int64', 'float64']).columns
print(non_num_columns)

Index([], dtype='object')


Standard Scaler

In [ ]:
#Selecting numerical columns
numerical_cols = merged_n.select_dtypes(include=['float64', 'int64']).columns

#Applying StandardScaler
standard_scaler = StandardScaler()
merged_n_standard = merged_n.copy()
merged_n_standard[numerical_cols] = standard_scaler.fit_transform(merged_n[numerical_cols])

MinMaxScaler

In [ ]:
minmax_scaler = MinMaxScaler()
merged_n_minmax = merged_n.copy()
merged_n_minmax[numerical_cols] = minmax_scaler.fit_transform(merged_n[numerical_cols])

Data Preparation

Train/Test Split

In [ ]:
end_of_october = merged_n.index.get_loc('2013-10-31').stop - 1
end_of_october

7127

In [ ]:
X = merged_n_standard.drop('Total Household Consumption (N)', axis=1).values
y = merged_n_standard['Total Household Consumption (N)'].values

#Data generation function
def generate_dataset(X, y, time_steps=168, out_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps - out_steps):
        v = X[i:i + time_steps]
        Xs.append(v)
        ys.append(y[i + time_steps:i + time_steps + out_steps])
    return np.array(Xs), np.array(ys)

TIME_STEPS = 24
OUT_STEPS = 1

X_data, y_data = generate_dataset(X, y, TIME_STEPS, OUT_STEPS)

split_index = end_of_october - TIME_STEPS + 1

X_train = X_data[:split_index]
X_test = X_data[split_index:]

y_train = y_data[:split_index]
y_test = y_data[split_index:]

LSTM Build-up

In [ ]:
def create_model(optimizer='adam', lstm_units=50, dropout_rate=0.0):
    model = Sequential()
    model.add(LSTM(lstm_units, activation='tanh', recurrent_activation='sigmoid',
                   input_shape=(TIME_STEPS, X_data.shape[2]), return_sequences=True, dropout=dropout_rate))
    model.add(LSTM(lstm_units, activation='tanh', recurrent_activation='sigmoid', dropout=dropout_rate))
    model.add(Dense(OUT_STEPS))
    model.compile(optimizer=optimizer, loss='mae')
    return model

In [ ]:
#KerasRegressor with parameters
regressor = KerasRegressor(build_fn=create_model, optimizer='adam', lstm_units=50, dropout_rate=0.0, verbose=0)

In [ ]:
#Defining the hyperparameters for grid search
param_grid = {
    'lstm_units': [50, 70],
    'dropout_rate': [0.0, 0.2],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [16, 32, 64],
    'epochs': [20, 30, 40]
}

In [ ]:
#Grid search setup
grid = GridSearchCV(estimator=regressor, param_grid=param_grid,
                    scoring='neg_mean_absolute_error', cv=2, verbose=1)

In [ ]:
grid_result = grid.fit(X_train, y_train)

Fitting 2 folds for each of 72 candidates, totalling 144 fits


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/scikeras

In [ ]:
#Grid Search results
print(f"Best score (negative MAE): {grid_result.best_score_}")
print(f"Best hyperparameters: {grid_result.best_params_}")

Best score (negative MAE): -0.19048355454501986
Best hyperparameters: {'batch_size': 16, 'dropout_rate': 0.0, 'epochs': 20, 'lstm_units': 50, 'optimizer': 'rmsprop'}


In [ ]:
import pickle

#Saving the grid search results
with open('grid_search_results.pkl', 'wb') as file:
    pickle.dump(grid_result, file)

In [ ]:
best_params = grid_result.best_params_
best_model = create_model(optimizer=best_params['optimizer'], lstm_units=best_params['lstm_units'], dropout_rate=best_params['dropout_rate'])
best_model.fit(X_train, y_train, batch_size=best_params['batch_size'], epochs=best_params['epochs'], verbose=1)
best_model.save('best_model.h5')


Epoch 1/20
444/444 [==============================] - 5s 5ms/step - loss: 0.2104
Epoch 2/20
444/444 [==============================] - 2s 5ms/step - loss: 0.1146
Epoch 3/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0937
Epoch 4/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0819
Epoch 5/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0744
Epoch 6/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0700
Epoch 7/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0662
Epoch 8/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0637
Epoch 9/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0609
Epoch 10/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0586
Epoch 11/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0569
Epoch 12/20
444/444 [==============================] - 2s 5ms/step - loss: 0.0558
Epoch 13/20
444/444 [====